In [1]:
import sys 

f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

In [2]:
f = open('labels.txt')
raw_labels = f.readlines()
f.close()

In [3]:
tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

In [4]:
print(tokens[50])

{'', 'declared', 'can', 'at', 'serious', 'man', 'aro', 'also', 'my', 'work', 'people', 'rate', 'personality', 'used', 'through', 'is', 'tolbukhin', 'on', 'that', 'into', 'him', 'to', 'and', 'so', 'testimony', 'not', 'got', 'he', 'fire', 'several', 'burnt', 'with', 'about', 'real', 'movie', 'murdered', 's', 'how', 'set', 'the', 'know', 'mission', 'actors', 'pregnant', 'alive', 'documentary', 'trauma', 'morbid', 'by', 'turn', 'horrifying', 'facts', 'in', 'kill', 'interviews', '.', 'this', 'played', 'br', 'them', 'another', 'then', 'some', 'scenes', '\n', 'analytical', 'childhood', 'women', 'monster', 'portraits', 'of', 'all', 'it', 'guatemala', 'a', 'based', 'had', 'seven', 'such'}


In [5]:
vocab = set()
for sent in tokens:
    for word in sent:
        if (len(word)>0):
            vocab.add(word)
#print(vocab)            
vocab = list(vocab)
#print(vocab)

In [6]:
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i
    
#print(word2index)    
    

In [8]:
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

In [9]:
#print(input_dataset[50])

In [10]:
target_dataset = []
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [11]:
import numpy as np 
np.random.seed(1)

def sigmoid(x):
    return 1/(1+np.exp(-x))

alpha = 0.01
iteration = 2 
hidden_size=100 

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1 


- len(vocab) = (74074,100) weight_0_1
- weight_0_2 = 100,1
- len(input_dataset) = 25000

In [12]:
correct,total = (0,0)
for iter in range(iteration):
    for i in range(len(input_dataset)-1000):
        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
        
        layer_2_delta = layer_2 - y
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)
        
        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha
        
        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)+' Progress:'+progress[2:4]+'.'+progress[4:6]+'% Training Accuracy:'+ str(correct/float(total)) + '%')


Iter:1 Progress:95.99% Training Accuracy:0.8662708333333333%%

In [13]:
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):
    x = input_dataset[i]
    y = target_dataset[i]
    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))

Test Accuracy:0.852


# Comparing Word Embeddings

In [14]:
from collections import Counter
import math

In [17]:
def similar(target = 'beautiful'):
    target_index = word2index[target]
    score = Counter()
    for word,index in word2index.items():
        raw_diff= weights_0_1[index] - weights_0_1[target_index]  #x2-x1
        squared_diff = raw_diff*raw_diff  # (x2-x2)^2
        score[word] = -math.sqrt(sum(squared_diff))
    return score.most_common(10)

In [18]:
print(similar('beautiful'))

[('beautiful', -0.0), ('atmosphere', -0.7006452818206494), ('powerful', -0.7148132486963501), ('unexpected', -0.7567074311369096), ('intense', -0.7623544213511201), ('each', -0.7935165303021007), ('flawless', -0.796508664109354), ('episode', -0.7968338322040082), ('impact', -0.7977518761533671), ('genre', -0.8005956766612187)]


In [19]:
print(similar('terrible'))

[('terrible', -0.0), ('boring', -0.7379198846703465), ('disappointment', -0.799035324868097), ('annoying', -0.8201206847170938), ('badly', -0.8266427362893946), ('poor', -0.8288579964200944), ('laughable', -0.8673699844148337), ('disappointing', -0.8796474034124275), ('dull', -0.8816469271123015), ('horrible', -0.8927423623462936)]
